<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       ModelOps Quickstart - Import your first BYOM in ModelOps 
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction:</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This is the notebook associated to quickstart of Bring-your-own-model (BYOM) of ModelOps v7 located at quickstart.teradata.com</p>
    
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>  This quickstart helps you to get started quickly using ClearScape Analytics ModelOps. We discuss key concepts briefly, so you can get right down to importing your first models into ModelOps.<br> <br> In this tutorial, you will learn:</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>What’s the difference between BYOM functions and ModelOps BYOM</li>
    <li>Importing your first BYOM model through the graphical user interface</li>
    <li>Deploying the model in Vantage with automated scheduling and monitoring capabilities</li>
</ul>

![image](images/byom_meth.png) 

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Steps in this Guide</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
  
<li> Create a project (ModelOps)</li>
<li> Environment Setup (Notebook)</li>
<li> Creating datasets (ModelOps)</li>
<li> Train a model and export to PMML (Notebook)</li>
<li> Import the PMML into Vantage using BYOM functions (Notebook) </li>
<li> Import the PMML into Vantage using ModelOps Graphical user interface (ModelOps) </li>
<li> Default and Custom alerting rules for Monitoring (ModelOps) </li>
<li> Custom Evaluation metrics and charts (Notebook)</li>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>As this is a complementary notebook to the quickstart, we will be covering the specific steps that are required to be done in a Notebook environment. For the ModelOps graphical user interface use the quickstart page as referece</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Create a project (ModelOps)</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Check the quickstart page to follow the steps</p>

<p style = 'font-size:16px;font-family:Arial'>If you are using clearscape.teradata.com, you can use this button for direct acccess to ModelOps. Use <b>demo_user</b> and your environment/host password:

[![image](images/launchModelOps.png)](/modelops)*

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Environment Setup (Notebook)</b></p>

<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>2.1 Db admin setup alternative </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A Bteq script in the data folder is provided "modelops.bteq" to execute in a clean Vantage environment these steps by a dbc to speed up this process, although we provide the steps to run this manually in the next step:</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Create user modelops</li>
    <li>Create database modelops</li>
    <li>Create table for byom models - aoa_byom_models</li>
    <li>Create table for metadata of statistics - aoa_statistics_metadata</li>
    <li>Create table for demo data - evaluation data - pima_patient_diagnoses </li>
    <li>Create table for demo data - features data - pima_patient_features</li>
    <li>Create table for demo data - output data - pima_patient_predictions </li>
    <li>Upload data for these tables</li>
    <li>Provide the rights to the login in Vantage for BYOM and VAL</li>
    </ul>

<hr style="height:2px;border:none;background-color:#00233C;"><p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>2.2 Manual environment setup alternative </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> In this section, we will Install libraries, Connect to Vantage, create the tables and import the data needed for the tutorial</p>

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2.2.1 Libraries Installation and Import </b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Install the required libraries and <b>Restart the kernel</b></p>

In [ ]:
%pip install -q teradataml==20.0.0.0 teradatamodelops==7.0.6 pandas==1.5.3

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Import the libraries</p>

In [ ]:
#teradatamodelops package contains aoa which is the SDK for ModelOps. We will use here the creation of statistics table
from aoa import create_features_stats_table

#teradataml is the package to use in Vantage
from teradataml import create_context, get_context, copy_to_sql, DataFrame, execute_sql
from teradatasqlalchemy.types import *
import pandas as pd
import getpass
import logging
import sys

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2.2.2 Connect to Vantage </b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Connect to your Vantage environment, you can use clearscape.teradata.com for a free environment</p>

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

host = input("Host:")
username = input("Username:")
password = getpass.getpass("Password:")

engine = create_context(host=host, username=username, password=password, logmech="TDNEGO")

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2.2.3 Create database objects for ModelOps </b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we create the following 2 very IMPORTANT tables:</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li>BYOM binary tables: aoa_byom_models - contains the binary of the BYOM models as BLOB in database    
<li>Statistics metadata table: aoa_feature_metadata - contains the metadata of features</li>
    
</ul>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> <b>Statistics metadata table</b> This table is special and <b>required</b> for ModelOps monitoring. This is the table where users will incorporate the histogram bins or edges for statistics computation, then ModelOps Agent will fill this table for every job executed. <b> IMPORTANT: user is required to compute this metadata the first time for each new dataset template </b>. Here we will show you how to create this table, and how to generate the metadata using different alternatives.</p>

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2.2.3.1 Create database objects for ModelOps with SQL Alternative</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>create byom models table:</p>

In [ ]:
query = '''
CREATE MULTISET TABLE aoa_byom_models
    (
        model_version VARCHAR(255),
        model_id VARCHAR(255),
        model_type VARCHAR(255),
        project_id VARCHAR(255),
        deployed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        model BLOB(2097088000)
    )
    UNIQUE PRIMARY INDEX ( model_version );
'''

try:
    execute_sql(query)
except:
    execute_sql('DROP TABLE aoa_byom_models;')
    execute_sql(query)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>create statistics metadata table:</p>

In [ ]:
query = '''
CREATE TABLE aoa_statistics_metadata (
      column_name VARCHAR(128), 
      column_type VARCHAR(128),
      stats JSON, 
      update_ts TIMESTAMP)
  UNIQUE PRIMARY INDEX ( column_name );
'''

try:
    execute_sql(query)
except:
    execute_sql('DROP TABLE aoa_statistics_metadata;')
    execute_sql(query)

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2.2.3.2 Create database objects for ModelOps with CLI Alternative</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For this, we need to open a terminal from Notebook like in the following picture:

<img src="images/terminal.png" alt="Evaluation" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>CLI of ModelOps is called tmo. This is installed when you install the teradatamodelops python library. In the beginning of this notebook you installed the required version for this quickstart. Let's confirm the version install by running:</p>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>run this in the CLI:</p>
    
    tmo --version

<img src="images/tmo_version.png" alt="Evaluation" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>CLI of ModelOps help
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>run this in the CLI:</p>
    
    tmo --help

<img src="images/tmo_help.png" alt="Evaluation" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Connect ModelOps to this CLI session by copying from ModelOps the session details. Go to your ModelOps environment and click on Your Account -> Session details. </p>

<img src="images/session.png" alt="Evaluation" width="200" height="200"/>

<img src="images/cli_config.png" alt="Evaluation" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Add a new connection for CLI
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>run this in the CLI:</p>
    
    tmo connection add

<img src="images/tmo_connection.png" alt="Evaluation" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now create the byom models table with this command
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>run this in the CLI:</p>
    
    tmo connection create-byom-table -n aoa_byom_models -e

<img src="images/aoa_byom_table.png" alt="Evaluation" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now create the statistics metadata table with this command
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>run this in the CLI:</p>
    
    tmo feature create-stats-table -m aoa_statistics_metadata -e

<img src="images/tmo_connection2.png" alt="Evaluation" width="500" height="500"/>

<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>2.2.4 Create Data tables for PIMA Diabetes demo model </b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we create 3 data tables, the first two tables will be created while importing the csv Data into Vantage:</p>

<li> Table `pima_patient_features` contains the features related to the patients medical history </li>
<li> Table `pima_patient_diagnoses` contains the diabetes diagnostic results for the patients.</li>
<li> Table `pima_patient_predictions` that will be used for storing the predictions (the output) of the model scoring</li>

In [ ]:
from teradataml import copy_to_sql, DataFrame
from teradatasqlalchemy.types import *
import pandas as pd

df = pd.read_csv("./data/pima_patient_features.csv")
copy_to_sql(df=df, 
            table_name="pima_patient_features",     
            schema_name=username,
            primary_index="PatientId", 
            if_exists="replace",  
            types={
                "PatientId": INTEGER,
                "NumTimesPrg": INTEGER, 
                "PlGlcConc": INTEGER,
                "BloodP": INTEGER,
                "SkinThick": INTEGER,
                "TwoHourSerIns": INTEGER,
                "BMI": FLOAT,
                "DiPedFunc": FLOAT,
                "Age": INTEGER
            })

In [ ]:
df = pd.read_csv("data/pima_patient_diagnoses.csv")
copy_to_sql(df=df, 
            table_name="pima_patient_diagnoses",     
            schema_name=username,
            primary_index="PatientId", 
            if_exists="replace",  
            types={
                "PatientId": INTEGER,
                "HasDiabetes": INTEGER
            })

In [ ]:
query = '''CREATE MULTISET TABLE Pima_Patient_Predictions 
     (
      job_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      PatientId BIGINT,
      HasDiabetes BIGINT,
      json_report CLOB(1048544000) CHARACTER SET LATIN)
PRIMARY INDEX ( job_id );;
'''
 
try:
    execute_sql(query)
except:
    execute_sql('DROP TABLE DEMO_USER.Pima_Patient_Predictions;')
    execute_sql(query)

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2.2.4 Compute Statistics for Demo data </b>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'> <b>Statistics metadata table</b> This table is special and <b>required</b> for ModelOps monitoring. This is the table where users will incorporate the histogram bins or edges for statistics computation, then ModelOps Agent will fill this table for every job executed. <b> IMPORTANT: user is required to compute this metadata the first time for each new dataset template </b>. Here we will show you how to create this table, and how to generate the metadata using different alternatives.</p>

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2.2.4.1 Import statistics metadata with SQL Alternative </b>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'> We provide you with the feature metadata statistics pre-computed for the demo model in a csv file at data folder. Load it with this command. <b>IMPORTANT in a new use case you will need to use the CLI to generate this</b></p>

In [ ]:
from teradataml import copy_to_sql, DataFrame
from teradatasqlalchemy.types import *
import pandas as pd

# we can compute this from the CLI also - but lets import pre-computed for now.
df = pd.read_csv("data/aoa_statistics_metadata.csv")
copy_to_sql(df=df, 
            table_name="aoa_statistics_metadata",     
            schema_name=username,
            if_exists="append")


<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>2.2.4.1 Import statistics metadata with CLI Alternative </b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Compute statistics in CLI</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You need to execute the CLI command for your <b>Continuous (Numeric)</b> features and another time for your <b>Categorical</b> features</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Check the usage using following command:</p>

    
        tmo feature compute-stats -h

<img src="images/tmo_feature_stats.png" alt="Evaluation" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Running metadata statistics for Continuous features in the demo model</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>run the following command <b>NOTE: YOU CAN USE --debug to see all log</b>:</p>

    
        tmo feature compute-stats --debug \
        -s demo_user.pima_patient_features \
        -m demo_user.aoa_statistics_metadata \
        -t continuous -c numtimesprg,plglcconc,bloodp,skinthick,twohourserins,bmi,dipedfunc,age

<img src="images/tmo_stats_continuous.png" alt="Evaluation" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Running metadata statistics for Categorical features in the demo model</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>run the following command:</p>

    
        tmo feature compute-stats --debug \
        -s demo_user.pima_patient_diagnoses \
        -m demo_user.aoa_statistics_metadata \
        -t categorical -c hasdiabetes

<img src="images/tmo_stats_categorical.png" alt="Evaluation" width="500" height="500"/>

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Creating Datasets (ModelOps)</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Check the quickstart page to follow the steps</p>

<p style = 'font-size:16px;font-family:Arial'>If you are using clearscape.teradata.com, you can use this button for direct acccess to ModelOps. Use <b>demo_user</b> and your environment/host password:

[![image](images/launchModelOps.png)](/modelops)*

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Train a model and export to PMML (Notebook)</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This section will cover the training of of the PIMA diabetes use case with the open source algorithm XGBoost and converting into PMML standard BYOM format. <b>PMML</b> stands for Predictive Model Markup Language. PMML provides a way for analytic applications to describe and exchange predictive models produced by data mining and machine learning algorithms. </p>

<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>4.1 Install and import libraries</b></p>


In [ ]:
%pip install -q xgboost scikit-learn nyoka skl2onnx onnxmltools

In [ ]:
import os
import pandas as pd
import getpass

from teradataml import (
    create_context, 
    remove_context,
    get_connection,
    DataFrame,
    PMMLPredict,
    configure)


<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>4.2 Train Model</b></p>

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

database = username

train_pdf = DataFrame.from_query(f"""
SELECT 
    F.*, D.hasdiabetes
FROM {database}.PIMA_PATIENT_FEATURES F 
JOIN {database}.PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 <> 0
""").to_pandas(all_rows=True)

features = ["NumTimesPrg", "Age", "PlGlcConc", "BloodP", "SkinThick", "TwoHourSerIns", "BMI", "DiPedFunc"]
target = "HasDiabetes"

# split data into X and y
X_train = train_pdf[features]
y_train = train_pdf[target]

model = Pipeline([('scaler', MinMaxScaler()),
                  ('xgb', XGBClassifier(eta=0.2, max_depth=6))])

model.fit(X_train, y_train)


<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>4.3 Convert Model to PMML</b></p>

In [ ]:
from nyoka import xgboost_to_pmml

xgboost_to_pmml(pipeline=model, col_names=features, target_name=target, pmml_f_name="model.pmml")

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Import the PMML into Vantage using BYOM functions (Notebook)</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This section will cover the BYOM package to show how users can <b>validate the formats and scoring output</b> from the Notebook. This option will not be registered in the Model Registry of ModelOps, and it won't be available for Governance, Automation or Monitoring ModelOps capabilities. to save BYOM, use save_byom function

In [ ]:
from teradataml import save_byom,list_byom,retrieve_byom,delete_byom,DataFrame

configure.byom_install_location = "mldb"
try:
    save_byom("pima_model",
              "model.pmml",
              "byom_models",
              additional_columns={},
              schema_name=username
    )
except:
    delete_byom(model_id="pima_model", table_name="byom_models")
    save_byom("pima_model",
              "model.pmml",
              "byom_models",
              additional_columns={},
              schema_name=username
    )


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>to list the byom model, use list_byom

In [ ]:
list_byom(table_name="byom_models")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>PMMLPredict to run model execution:</p>

In [ ]:
result = PMMLPredict(
 modeldata = retrieve_byom("pima_model", "byom_models"),
 newdata = DataFrame.from_query("SELECT * FROM Pima_Patient_Features WHERE patientid MOD 5 = 0"),
 accumulate = ['PatientId'],
 overwrite_cached_models = '*',
 )

print(result.show_query())

result.result

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>6. Import the PMML into Vantage using ModelOps Graphical user interface (ModelOps)</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Check the quickstart page to follow the steps</p>

<p style = 'font-size:16px;font-family:Arial'>If you are using clearscape.teradata.com, you can use this button for direct acccess to ModelOps. Use <b>demo_user</b> and your environment/host password:

[![image](images/launchModelOps.png)](/modelops)*

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>7. Go through Automated Lifecycle - Evaluation, Approve, Deploy (ModelOps)</b></p>

<p style = 'font-size:16px;font-family:Arial'>If you are using clearscape.teradata.com, you can use this button for direct acccess to ModelOps. Use <b>demo_user</b> and your environment/host password:

[![image](images/launchModelOps.png)](/modelops)*

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>8. Default and Custom alerting rules for Monitoring (ModelOps)</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Check the quickstart page to follow the steps</p>

<p style = 'font-size:16px;font-family:Arial'>If you are using clearscape.teradata.com, you can use this button for direct acccess to ModelOps. Use <b>demo_user</b> and your environment/host password:

[![image](images/launchModelOps.png)](/modelops)*

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>9. Custom Evaluation metrics and charts (Notebook)</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You can define custom evaluation logic for BYOM models in ModelOps. This allows you to define your own charts, metrics etc that are to be created and captured as part of evaluation / comparison.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>With this logic you will generate your own evaluation.py and data_stats.json that you can include in your Import wizard in ModelOps graphical user interface</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Logic for compute statistics of Dataset and export into data_stats</p>

In [ ]:
from aoa.stats.stats import _capture_stats, _NpEncoder
import json
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

configure.val_install_location = "VAL"
configure.byom_install_location = "MLDB"

train_df = DataFrame.from_query("""
SELECT 
    F.*, D.hasdiabetes 
FROM Pima_Patient_Features F
JOIN pima_patient_diagnoses D
    ON F.patientid = D.patientid 
    WHERE F.patientid MOD 5 <> 0
""")

data_stats = _capture_stats(df=train_df,
                            features=features,
                            targets=[target],
                            categorical=[target],
                            feature_metadata_fqtn=f"{username}.aoa_statistics_metadata")

with open("data_stats.json", 'w+') as f:
    json.dump(data_stats, f, indent=2, cls=_NpEncoder)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Evaluate standard logic, here you can make changes of metrics and charts</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The evaluation function takes the following shape</p>

```python
def evaluate(context: ModelContext, **kwargs):
    aoa_create_context()

    # read your model from Vantage
    model = DataFrame(f"model_${context.model_version}")
    
    # your evaluation logic
    
    record_evaluation_stats(...)
```
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You can execute this from the CLI or directly within the notebook as shown.</p>

In [ ]:
%%writefile ./evaluation.py

from sklearn import metrics
from teradataml import (
    get_context,
    DataFrame,
    PMMLPredict,
    configure
)
from aoa import (
    record_evaluation_stats,
    aoa_create_context,
    store_byom_tmp,
    ModelContext
)

import os
import json


def plot_confusion_matrix(cf, img_filename):
    import itertools
    import matplotlib.pyplot as plt
    plt.imshow(cf, cmap=plt.cm.Blues, interpolation='nearest')
    plt.colorbar()
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.xticks([0, 1], ['0', '1'])
    plt.yticks([0, 1], ['0', '1'])

    thresh = cf.max() / 2.
    for i, j in itertools.product(range(cf.shape[0]), range(cf.shape[1])):
        plt.text(j, i, format(cf[i, j], 'd'), horizontalalignment='center',
                 color='white' if cf[i, j] > thresh else 'black')

    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()


def evaluate(context: ModelContext, **kwargs):
    aoa_create_context()

    
    # this evaluation.py can hanlde both onnx and pmml. usually, you would only need to support one but for 
    # demo purposes, we will show with both as we produce both onnx and pmml in this notebook.
    
    import glob
    for file_name in glob.glob(f"{context.artifact_input_path}/model.*"):
        model_type = file_name.split(".")[-1]
    
    with open(f"{context.artifact_input_path}/model.{model_type}", "rb") as f:
        model_bytes = f.read()
        
    model = store_byom_tmp(get_context(), "byom_models_tmp", context.model_version, model_bytes)

    target_name = context.dataset_info.target_names[0]

    if model_type.upper() == "ONNX":
        byom_target_sql = "CAST(CAST(json_report AS JSON).JSONExtractValue('$.output_label[0]') AS INT)"
        mldb = os.environ.get("AOA_BYOM_INSTALL_DB", "MLDB")

        query = f"""
            SELECT sc.{context.dataset_info.entity_key}, {target_name}, sc.json_report
                FROM {mldb}.ONNXPredict(
                    ON ({context.dataset_info.sql}) AS DataTable
                    ON (SELECT model_version as model_id, model FROM byom_models_tmp) AS ModelTable DIMENSION
                    USING
                        Accumulate('{context.dataset_info.entity_key}', '{target_name}')
            ) sc;
        """

        predictions_df = DataFrame.from_query(query)
        
    elif model_type.upper() == "PMML":
        byom_target_sql = "CAST(CAST(json_report AS JSON).JSONExtractValue('$.predicted_HasDiabetes') AS INT)"
        
        pmml = PMMLPredict(
            modeldata=model,
            newdata=DataFrame.from_query(context.dataset_info.sql),
            accumulate=[context.dataset_info.entity_key, target_name])
        
        predictions_df = pmml.result

    predictions_df.to_sql(table_name="predictions_tmp", if_exists="replace", temporary=True)

    metrics_df = DataFrame.from_query(f"""
    SELECT 
        HasDiabetes as y_test, 
        {byom_target_sql} as y_pred
        FROM predictions_tmp
    """)
    metrics_df = metrics_df.to_pandas()

    y_pred = metrics_df[["y_pred"]]
    y_test = metrics_df[["y_test"]]

    evaluation = {
        'Accuracy': '{:.2f}'.format(metrics.accuracy_score(y_test, y_pred)),
        'Recall': '{:.2f}'.format(metrics.recall_score(y_test, y_pred)),
        'Precision': '{:.2f}'.format(metrics.precision_score(y_test, y_pred)),
        'f1-score': '{:.2f}'.format(metrics.f1_score(y_test, y_pred))
    }

    with open(f"{context.artifact_output_path}/metrics.json", "w+") as f:
        json.dump(evaluation, f)

    # create confusion matrix plot
    cf = metrics.confusion_matrix(y_test, y_pred)

    plot_confusion_matrix(cf, f"{context.artifact_output_path}/confusion_matrix")

    # calculate stats if training stats exist
    if os.path.exists(f"{context.artifact_input_path}/data_stats.json"):
        record_evaluation_stats(features_df=DataFrame.from_query(context.dataset_info.sql),
                                predicted_df=DataFrame("predictions_tmp"),
                                context=context)


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>ModelOps SDK code to run the evaluation logic at the Notebook level for debugging, <b>this is not storing the metadata in ModelOps</b>, only remains at Notebook level. Later you can download the evaluation.py the logic and include it into the BYOM imported model to run it in ModelOps</p>

In [ ]:
from aoa import ModelContext, DatasetInfo

# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the evaluation dataset 
sql = """
SELECT 
    F.*, D.hasdiabetes 
FROM PIMA_PATIENT_FEATURES F 
JOIN PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 = 0
"""

feature_metadata =  {
    "database": database,
    "table": "aoa_statistics_metadata"
}

entity_key = "PatientId"
target_names = ["HasDiabetes"]
feature_names = ["NumTimesPrg", "PlGlcConc", "BloodP", "SkinThick", "TwoHourSerIns", "BMI", "DiPedFunc", "Age"]

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata)

ctx = ModelContext(hyperparams={},
                   dataset_info=dataset_info,
                   artifact_output_path="/tmp",
                   artifact_input_path="./",
                   model_version="v1",
                   model_table="aoa_model_v1")


# drop volatile table from session if executing multiple times
try:
    get_context().execute(f"DROP TABLE byom_models_tmp")
except: 
    pass

import evaluation
evaluation.evaluate(context=ctx)

# view evaluation results
with open(f"{ctx.artifact_output_path}/metrics.json") as f:
    print(json.load(f))

In [ ]:
DataFrame.from_query("SELECT PatientId, HasDiabetes, json_report FROM predictions_tmp")

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>